# Marketing Mix Modeling in Python

![banner](https://raw.githubusercontent.com/MayumyCH/marketing_mix_modeling_project_python/main/resources/banner.jpeg)

_Este proyecto propone el análisis de la data proporcionada por Kaggle para el área de Marketing; se busca encontrar **"Qué inversión publicitaria impulsa** realmente **las ventas"**._

In [15]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 

In [16]:
# Importar los datos
url_data = "dataset/data_marketing.csv"
data_Marketing = pd.read_csv(url_data)
# data_Marketing.tail()
data_Marketing.head()

,TV,Radio,Social Media,Influencer,Sales
0,16.0,6.566231,2.907983,Mega,54.732757
1,13.0,9.237765,2.409567,Mega,46.677897
2,41.0,15.886446,2.913410,Mega,150.177829
3,83.0,30.020028,6.922304,Mega,298.246340
4,15.0,8.437408,1.405998,Micro,56.594181


In [17]:
data_Marketing.shape

# INTERPRETACION:
# 4572 observaciones
# 5 features/variables

(4572, 5)

In [27]:
# CANTIDAD DE NULOS POR CADA FEATURE
data_Marketing.isnull().sum()

# INTERPRETACION:
# Variable Tv: 10 Valores nulos
# Variable Radio: 6 Valores nulos
# Variable Social: 6 Valores nulos
# Variable Sales(Ventas): 6 Valores nulos

TV              0
Radio           0
Social Media    0
Influencer      0
Sales           0
dtype: int64

In [26]:
data_Marketing.info()

# INTERPRETACION:
# 5 Features tipo Numericos (float)
# 1 Features tipo Categorico (object)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TV            4572 non-null   float64
 1   Radio         4572 non-null   float64
 2   Social Media  4572 non-null   float64
 3   Influencer    4572 non-null   object 
 4   Sales         4572 non-null   float64
dtypes: float64(4), object(1)
memory usage: 178.7+ KB


In [20]:
data_Marketing.describe().T 
# Nota en este caso solo se veran los describes de las variables numericas!

,count,mean,std,min,25%,50%,75%,max
TV,4562.0,54.066857,26.125054,10.000000,32.000000,53.000000,77.000000,100.000000
Radio,4568.0,18.160356,9.676958,0.000684,10.525957,17.859513,25.649730,48.871161
Social Media,4566.0,3.323956,2.212670,0.000031,1.527849,3.055565,4.807558,13.981662
Sales,4566.0,192.466602,93.133092,31.199409,112.322882,189.231172,272.507922,364.079751


## Analisis exploratorios ( EDA)

In [25]:
## En este caso como hay valores nulos procederemos a trabajar a llenar estos valores
## Como los valores que faltan son muy pequenos podria proceder a eliminarlos
data_Marketing['TV'].fillna(data_Marketing['TV'].median(), inplace=True)
data_Marketing['Radio'].fillna(data_Marketing['Radio'].median(), inplace=True)
data_Marketing['Social Media'].fillna(data_Marketing['Social Media'].median(), inplace=True)
data_Marketing['Sales'].fillna(data_Marketing['Sales'].median(), inplace=True)

## Regresión Lineal Multiple
Representante más simple de los modelos aditivos!

Nuestra regresión multiple se veria de la sgte forma:

> $Y_{Sales} = Base + a(X_{Tv}) + b( X_{Radio}) + c( X_{Banner}$)


In [28]:
target = 'Sales'
predictoras = ['TV','Radio','Social Media']

In [29]:
X = data_Marketing[predictoras]
y = data_Marketing[target]

In [30]:
# Generamos el conjunto de train y test gracias a sklearn
from sklearn.model_selection import train_test_split, cross_val_score
# Importamos el Modelo de Regresion Lineal
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
# Creamos el modelo de la regresion
lm = LinearRegression() # Creando un objeto de Regresión Lineal 'lm'
# Entrenamiento del modelo
lm.fit(X_train, y_train)

LinearRegression()

In [31]:
print("La intersección del modelo lineal:", lm.intercept_)
print("Los coeficientes del modelo lineal:", lm.coef_)

La intersección del modelo lineal: 0.3902946338252775
Los coeficientes del modelo lineal: [3.50398802 0.14088139 0.02341146]


El modelo quedara del sgte modo:
> $Y_{Sales} = 0.39 + 3.5(X_{Tv}) + 0.14( X_{Radio}) + 0.02( X_{Banner}$)

**Interpretación:**

*   Si no invertimos en publicidad en radio podremos decir que: Por cada aumento en una unidad de **la inversion en publicidad en la Tv**, el **total de ventas del producto** aumentará en **0.06 mil de soles**.
*   Si no invertimos en publicidad en tv podremos decir que: Por cada aumento en una unidad de **la inversion en publicidad en la radio**, el **total de ventas del producto** aumentará en **0.10 mil de soles**.
*   Cuando las variables predictoras sean 0, el **total de ventas del producto** es de 4.48 mil de soles


In [45]:
# Predecimos la data de entrenamiento y la data del test
train_pred=lm.predict(X_train)
test_pred=lm.predict(X_test)

In [52]:
# Visualizaremos el ENTRENAMIENTO y la PREDICCION
# -----------------------------------------------
df_train= pd.concat([X_train, y_train], axis = 1).head() # DATA REAL DE ENTRENAMIENTO
df_train['Sales Predict'] = train_pred[:5] # LA PREDICIÓN DE ENTRENAMIENTO
df_train

,TV,Radio,Social Media,Sales,Sales Predict
2546,20.0,3.315551,0.424423,71.754233,70.947091
732,25.0,0.981140,1.377660,92.302431,88.160473
3559,86.0,18.907308,4.165040,305.204371,304.494462
1729,52.0,19.521404,5.296023,188.037471,185.471862
618,78.0,18.159047,4.246496,274.093997,276.359049


El modelo quedara del sgte modo:
-- 2546--
> $Y_{Sales} = 0.39 + 3.5(20.0) + 0.14( 3.32) + 0.02( 0.42)$

In [53]:
# Ejemplo manual # 2546
y_2546 = 0.3902946338252775 + 3.50398802*(20.0) + 0.14088139*(3.315551) + 0.02341146*(0.424423)
y_2546

70.94709082940875

In [56]:
71.754233 - 70.94709082940875

0.8071421705912485

In [63]:
71.754233 / 70.94709082940875

1.0113766774811388

Como se observa de lo predecido al valor real existe una diferencia de 0.807

In [58]:
3.50398802*(20.0)
# Contribucion no ajustada de la television

70.0797604

> $coef_{Tv} * (X_{Tv}) = 3.50398802(20) =70.0797604$

In [ ]:
# multipliquemos las contribuciones y la línea de base 
# por un factor de corrección correct_factor = 15904.11 / 16147.0159 ≈ 0.985 y todo está bien. Obtenemos

> $correctFactor = \frac{Y_{sales}}{Y_{salesPredict}} = \frac{71.754233}{70.94709082940875}  \approx 1.011$

Entonces la contribucion de la Tv es:
    
> $Contribucion_{Tv} = correctFactor * (70.0797604) = 1.011* (70.0797604) =70.85063776439999$

In [67]:
1.011* (70.0797604)

70.85063776439999